In [14]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

url_target = 'https://standings.stalruth.dev/2025/international-latin-america/masters'
page = urlopen(url_target)
soup = BeautifulSoup(page)

In [2]:
standing_table = soup.findAll('table')[1]

In [3]:
standing_rows = standing_table.findAll('tr')

In [4]:
standing_rows[0].find('td', attrs={'class': 'name-cell'})

<td class="name-cell">
<a href="/2025/international-latin-america/masters/201">
              Marco Hemantha Kaludura Silva [IT]
            </a>
</td>

In [34]:
mons_usage_wr_dict = {}


In [33]:
drop_form_names = ['Sinistcha', 'Tatsugiri', 'Maushold']

In [35]:
for r in standing_rows:
    mons_ = r.find_all('span', attrs={'class': 'pokemon-icon'})
    record_ = r.find('td', attrs={'class': 'record-resistance'}).find('p', attrs={'class': 'record'}).get_text()
    for m in mons_:
        mon_name = m.get('title')
        # Drop the form names
        for dpn in drop_form_names:
            if dpn in mon_name:
                mon_name = dpn
        if not (mon_name in mons_usage_wr_dict.keys()):
            mons_usage_wr_dict[mon_name] = {'win': int(record_.split('-')[0]),
                                    'loss': int(record_.split('-')[1]),
                                    'day1': 1,
                                    'day2': 1 if (int(record_.split('-')[0]) > 5) else 0
                                    }
        else:
            mons_usage_wr_dict_temp = {'win': mons_usage_wr_dict[mon_name]['win'] + int(record_.split('-')[0]),
                                        'loss': mons_usage_wr_dict[mon_name]['loss'] + int(record_.split('-')[1]),
                                        'day1': mons_usage_wr_dict[mon_name]['day1'] + 1,
                                        'day2': mons_usage_wr_dict[mon_name]['day2'] + (1 if (int(record_.split('-')[0]) > 5) else 0)
                                        }
            mons_usage_wr_dict[mon_name] = mons_usage_wr_dict_temp


In [36]:
mons_usage_wr_dict

{'Ursaluna': {'win': 338, 'loss': 323, 'day1': 85, 'day2': 11},
 'Jumpluff': {'win': 44, 'loss': 45, 'day1': 11, 'day2': 2},
 'Charizard': {'win': 61, 'loss': 61, 'day1': 15, 'day2': 1},
 'Torkoal': {'win': 154, 'loss': 175, 'day1': 43, 'day2': 3},
 'Incineroar': {'win': 599, 'loss': 588, 'day1': 147, 'day2': 31},
 'Porygon2': {'win': 222, 'loss': 233, 'day1': 59, 'day2': 9},
 'Archaludon': {'win': 486, 'loss': 465, 'day1': 120, 'day2': 20},
 'Dragonite': {'win': 619, 'loss': 559, 'day1': 147, 'day2': 25},
 'Pelipper': {'win': 362, 'loss': 364, 'day1': 92, 'day2': 15},
 'Sinistcha': {'win': 127, 'loss': 101, 'day1': 28, 'day2': 7},
 'Gholdengo': {'win': 648, 'loss': 583, 'day1': 150, 'day2': 29},
 'Kommo-o': {'win': 46, 'loss': 34, 'day1': 10, 'day2': 3},
 'Sneasler': {'win': 822, 'loss': 783, 'day1': 204, 'day2': 32},
 'Rillaboom': {'win': 665, 'loss': 663, 'day1': 170, 'day2': 27},
 'Dondozo': {'win': 163, 'loss': 126, 'day1': 36, 'day2': 7},
 'Electabuzz': {'win': 323, 'loss': 281, 

In [37]:
mons_usage_winrate_df = pd.DataFrame(mons_usage_wr_dict).T

In [38]:
mons_usage_winrate_df

,win,loss,day1,day2
Ursaluna,338,323,85,11
Jumpluff,44,45,11,2
Charizard,61,61,15,1
Torkoal,154,175,43,3
Incineroar,599,588,147,31
...,...,...,...,...
Goodra-Hisui,1,7,1,0
Lapras,0,1,1,0
Staraptor,0,1,1,0
Fuecoco,0,2,1,0


In [39]:
mons_usage_winrate_df['win_rate'] = mons_usage_winrate_df['win']/(mons_usage_winrate_df['win'] + mons_usage_winrate_df['loss'])

In [40]:
mons_usage_winrate_df['conversion_rate(day2:day1)'] = (mons_usage_winrate_df['day2']/65) / (mons_usage_winrate_df['day1']/455)

In [41]:
mons_usage_winrate_df['day1']

Ursaluna         85
Jumpluff         11
Charizard        15
Torkoal          43
Incineroar      147
               ... 
Goodra-Hisui      1
Lapras            1
Staraptor         1
Fuecoco           1
Umbreon           1
Name: day1, Length: 126, dtype: int64

In [42]:
mons_usage_winrate_df = mons_usage_winrate_df.round(4)

In [43]:
mons_usage_winrate_df

,win,loss,day1,day2,win_rate,conversion_rate(day2:day1)
Ursaluna,338,323,85,11,0.5113,0.9059
Jumpluff,44,45,11,2,0.4944,1.2727
Charizard,61,61,15,1,0.5000,0.4667
Torkoal,154,175,43,3,0.4681,0.4884
Incineroar,599,588,147,31,0.5046,1.4762
...,...,...,...,...,...,...
Goodra-Hisui,1,7,1,0,0.1250,0.0000
Lapras,0,1,1,0,0.0000,0.0000
Staraptor,0,1,1,0,0.0000,0.0000
Fuecoco,0,2,1,0,0.0000,0.0000


In [44]:
mons_usage_winrate_df.to_csv('output_txt/laic_mons_summary.csv')

In [16]:
sneaslers_players = []
for r in standing_rows:
    if (r.find('span', title='Sneasler') != None):
        name_cell = r.find('td', attrs={'class': 'name-cell'})
        name_ = name_cell.get_text()
        link_ = name_cell.find('a').get('href')
        record_ = r.find('td', attrs={'class': 'record-resistance'}).find('p', attrs={'class': 'record'}).get_text()
        player_dict = {'player_name': name_,
                        'schedule_link_suffix': link_,
                        'record': record_,
                        'win': int(record_.split('-')[0]),
                        'loss': int(record_.split('-')[1])}
        sneaslers_players.append(player_dict)
        # sneaslers_players.append(r)

In [17]:
len(sneaslers_players)

193

In [18]:
positive_records = []
for sneasler_player in sneaslers_players:
    if sneasler_player['win'] >= 5:
        positive_records.append(sneasler_player)

In [19]:
positive_records

[{'player_name': '\n\n              Mattie Morgan [IE]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/341',
  'record': '13-3',
  'win': 13,
  'loss': 3},
 {'player_name': '\n\n              Nicolò Berardo [IT]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/153',
  'record': '12-3',
  'win': 12,
  'loss': 3},
 {'player_name': '\n\n              Ibrahim Maarouf [DE]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/328',
  'record': '11-3',
  'win': 11,
  'loss': 3},
 {'player_name': '\n\n              Nico Davide Cognetta [IT]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/161',
  'record': '11-3',
  'win': 11,
  'loss': 3},
 {'player_name': '\n\n              Joan Garcia [ES]\n            \n',
  'schedule_link_suffix': '/2025/regional-dortmund/masters/450',
  'record': '11-2',
  'win': 11,
  'loss': 2},
 {'player_name': '\n\n              Michael Kelsch [DE]\n          

Conversion rate of Sneasler: 193/710 -> 85/254 (5Wins+) WTF

In [20]:
sneasler_players_loses = {}
sneasler_players_allmatches = {}
url_prefix = 'https://standings.stalruth.dev'
for player in positive_records:
    player_schedule_link = url_prefix + player['schedule_link_suffix']
    print(player_schedule_link)
    page_ = urlopen(player_schedule_link)
    soup_ = BeautifulSoup(page_)
    tables = soup_.find_all('table')
    for t in tables:
        rows_ = t.find_all('tr')
        for row_ in rows_:
            win_or_loss = row_.find('td', attrs={'class': 'result-cell'}).get_text().strip("\n").strip()
            if (win_or_loss == 'Loss'):
                team_ = row_.find('td', attrs={'class': 'name-team'}).find('div', attrs={'class': 'team'}).find('a')
                # print(team_)
                team_mons_ = team_.find_all('span')
                # print(team_mons_)
                for mon_ in team_mons_:
                    print(mon_)
                    mon_name_ = mon_.get('title')
                    # print(mon_name_)
                    if mon_name_ in sneasler_players_loses.keys():
                        sneasler_players_loses[mon_name_] = sneasler_players_loses[mon_name_] + 1
                    else:
                        sneasler_players_loses[mon_name_] = 1


https://standings.stalruth.dev/2025/regional-dortmund/masters/341
<span class="pokemon-icon" style="background-position: -280px -2430px" title="Annihilape"></span>
<span class="pokemon-icon" style="background-position: -280px -1350px" title="Whimsicott"></span>
<span class="pokemon-icon" style="background-position: -40px -1110px" title="Garchomp"></span>
<span class="pokemon-icon" style="background-position: -120px -3510px" title="Clefable"></span>
<span class="pokemon-icon" style="background-position: -440px -1560px" title="Hydreigon"></span>
<span class="pokemon-icon" style="background-position: -160px -2490px" title="Gholdengo"></span>
<span class="pokemon-icon" style="background-position: -160px -1950px" title="Kommo-o"></span>
<span class="pokemon-icon" style="background-position: -240px -1050px" title="Drifblim"></span>
<span class="pokemon-icon" style="background-position: -440px -1560px" title="Hydreigon"></span>
<span class="pokemon-icon" style="background-position: -360px -21

In [21]:
sneasler_players_loses

{'Annihilape': 43,
 'Whimsicott': 29,
 'Garchomp': 46,
 'Clefable': 16,
 'Hydreigon': 18,
 'Gholdengo': 89,
 'Kommo-o': 14,
 'Drifblim': 3,
 'Toxtricity': 3,
 'Volcarona': 15,
 'Indeedee': 10,
 'Magmar': 27,
 'Kingambit': 95,
 'Rillaboom': 104,
 'Primarina': 88,
 'Dragapult': 54,
 'Sneasler': 87,
 'Porygon2': 40,
 'Incineroar': 88,
 'Vivillon': 16,
 'Sinistcha': 21,
 'Tyranitar': 20,
 'Excadrill': 8,
 'Talonflame': 16,
 'Dondozo': 24,
 'Glimmora': 10,
 'Pawmot': 8,
 'Tatsugiri': 14,
 'Kilowattrel': 4,
 'Weezing-Galar': 8,
 'Gallade': 9,
 'Corviknight': 7,
 'Ursaluna': 34,
 'Amoonguss': 62,
 'Flamigo': 20,
 'Typhlosion-Hisui': 14,
 'Toedscruel': 3,
 'Ursaluna-Bloodmoon': 18,
 'Farigiraf': 10,
 'Pelipper': 34,
 'Maushold-Four': 27,
 'Archaludon': 46,
 'Grimmsnarl': 9,
 'Gengar': 2,
 'Ninetales-Alola': 14,
 'Gastrodon': 8,
 'Electabuzz': 46,
 'Basculegion': 39,
 'Inteleon': 1,
 'Smeargle': 1,
 'Murkrow': 5,
 'Salamence': 14,
 'Chandelure': 2,
 'Barraskewda': 1,
 'Hydrapple': 1,
 'Maushold

In [24]:
import pandas as pd

df_count = pd.DataFrame(sneasler_players_loses.items(), columns = ['mon', 'count'])

In [35]:
df_count_sorted = df_count.sort_values(by='count', ascending=False).reset_index(drop=True)

In [38]:
pd.set_option('display.max_rows', 100)

In [39]:
df_count_sorted

,mon,count
0,Rillaboom,104
1,Kingambit,95
2,Gholdengo,89
3,Incineroar,88
4,Primarina,88
5,Sneasler,87
6,Amoonguss,62
7,Dragapult,54
8,Electabuzz,46
9,Garchomp,46


In [37]:
df_count_sorted[df_count_sorted['mon'] == 'Sneasler']

,mon,count
5,Sneasler,87
